In [1]:
#import packages
import schwabdev
import logging
import dotenv
import os

In [2]:
# set logging level
logging.basicConfig(level=logging.INFO)

#load environment variables and make client
dotenv.load_dotenv()
client = schwabdev.Client(os.getenv('app_key'), os.getenv('app_secret'), os.getenv('callback_url'))

INFO:Schwabdev.Tokens:Access token expires in 00H:26M:47S
INFO:Schwabdev.Tokens:Refresh token expires in 167H:56M:47S
INFO:Schwabdev.Client:Client Initialization Complete


# Basic API calls

In [3]:
# get account number and hashes for linked accounts
linked_accounts = client.account_linked().json()
print(linked_accounts)
# select the first account to use for orders
account_hash = linked_accounts[0].get('hashValue')

[{'accountNumber': '64577721', 'hashValue': '8FEC8181C0DCAB0A44C1BB39C924CBE5605B97DD6EE498389C24D8A31341E95D'}]


In [4]:
# get positions for selected account
print(client.account_details(account_hash, fields="positions").json())

{'securitiesAccount': {'type': 'MARGIN', 'accountNumber': '64577721', 'roundTrips': 0, 'isDayTrader': False, 'isClosingOnlyRestricted': False, 'pfcbFlag': False, 'positions': [{'shortQuantity': 0.0, 'averagePrice': 44.9578, 'currentDayProfitLoss': 32.0, 'currentDayProfitLossPercentage': 0.16, 'longQuantity': 500.0, 'settledLongQuantity': 500.0, 'settledShortQuantity': 0.0, 'instrument': {'assetType': 'EQUITY', 'cusip': '015393101', 'symbol': 'ALFVY', 'netChange': 0.064}, 'marketValue': 20547.0, 'maintenanceRequirement': 0.0, 'averageLongPrice': 44.9578, 'taxLotAverageLongPrice': 44.9578, 'longOpenProfitLoss': -1931.899999999999, 'previousSessionLongQuantity': 500.0, 'currentDayCost': 0.0}, {'shortQuantity': 0.0, 'averagePrice': 274.797861129081, 'currentDayProfitLoss': -97.424112000008, 'currentDayProfitLossPercentage': -0.09, 'longQuantity': 202.9669, 'settledLongQuantity': 202.9669, 'settledShortQuantity': 0.0, 'instrument': {'assetType': 'COLLECTIVE_INVESTMENT', 'cusip': '46090E103'

In [5]:
# get a list of quotes
print(client.quotes(["AAPL", "AMD"]).json())

{'AAPL': {'assetMainType': 'EQUITY', 'assetSubType': 'COE', 'quoteType': 'NBBO', 'realtime': True, 'ssid': 1973757747, 'symbol': 'AAPL', 'extended': {'askPrice': 233.9, 'askSize': 424, 'bidPrice': 233.62, 'bidSize': 295, 'lastPrice': 233.89, 'lastSize': 10, 'mark': 233.89, 'quoteTime': 1736911143000, 'totalVolume': 0, 'tradeTime': 1736911052000}, 'fundamental': {'avg10DaysVolume': 42588274.0, 'avg1YearVolume': 56859565.0, 'declarationDate': '2024-10-31T04:00:00Z', 'divAmount': 1.0, 'divExDate': '2024-11-08T05:00:00Z', 'divFreq': 4, 'divPayAmount': 0.25, 'divPayDate': '2024-11-14T05:00:00Z', 'divYield': 0.42221, 'eps': 6.08, 'fundLeverageFactor': 0.0, 'lastEarningsDate': '2024-10-31T04:00:00Z', 'nextDivExDate': '2025-02-10T05:00:00Z', 'nextDivPayDate': '2025-02-14T05:00:00Z', 'peRatio': 38.55263}, 'quote': {'52WeekHigh': 260.1, '52WeekLow': 164.075, 'askMICId': 'XNAS', 'askPrice': 233.78, 'askSize': 1, 'askTime': 1736902444863, 'bidMICId': 'ARCX', 'bidPrice': 233.43, 'bidSize': 5, 'bidT

In [6]:
# get an option chain
print(client.option_expiration_chain("AAPL").json())

{'expirationList': [{'expirationDate': '2025-01-17', 'daysToExpiration': 3, 'expirationType': 'S', 'settlementType': 'P', 'optionRoots': 'AAPL', 'standard': True}, {'expirationDate': '2025-01-24', 'daysToExpiration': 10, 'expirationType': 'W', 'settlementType': 'P', 'optionRoots': 'AAPL', 'standard': True}, {'expirationDate': '2025-01-31', 'daysToExpiration': 17, 'expirationType': 'W', 'settlementType': 'P', 'optionRoots': 'AAPL', 'standard': True}, {'expirationDate': '2025-02-07', 'daysToExpiration': 24, 'expirationType': 'W', 'settlementType': 'P', 'optionRoots': 'AAPL', 'standard': True}, {'expirationDate': '2025-02-14', 'daysToExpiration': 31, 'expirationType': 'W', 'settlementType': 'P', 'optionRoots': 'AAPL', 'standard': True}, {'expirationDate': '2025-02-21', 'daysToExpiration': 38, 'expirationType': 'S', 'settlementType': 'P', 'optionRoots': 'AAPL', 'standard': True}, {'expirationDate': '2025-02-28', 'daysToExpiration': 45, 'expirationType': 'W', 'settlementType': 'P', 'optionR

# Order example

In [7]:
# place an order for INTC at limit price $10.00
order = {"orderType": "LIMIT", 
         "session": "NORMAL", 
         "duration": "DAY", 
         "orderStrategyType": "SINGLE", 
         "price": '10.00',
         "orderLegCollection": [
             {"instruction": "BUY", 
              "quantity": 1, 
              "instrument": 
                  {"symbol": "INTC", 
                   "assetType": "EQUITY"
                   }
              }
         ]}
resp = client.order_place(account_hash, order)
print(f"Response code: {resp}") 

# get the order ID - if order is immediately filled then the id might not be returned
order_id = resp.headers.get('location', '/').split('/')[-1] 
print(f"Order id: {order_id}")

Response code: <Response [429]>
Order id: 


In [8]:
# cancel the order
print(client.order_cancel(account_hash, order_id))

<Response [429]>


# Streaming example

In [3]:
# create streamer
streamer = client.stream

In [4]:
# create a list to store responses
responses = []
def add_to_list(message):
    responses.append(message)

In [5]:
#start stream and send request
streamer.start(add_to_list)



INFO:Schwabdev.Stream:Connecting to streaming server...
INFO:Schwabdev.Stream:Connected to streaming server.


In [6]:

streamer.send(streamer.level_one_equities("AMD", "0,1,2,3,4,5,6,7,8"))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [7]:
#check responses
print(responses)

['{"response":[{"service":"ADMIN","command":"LOGIN","requestid":"1","SchwabClientCorrelId":"ee2e8716-4d94-dc8c-483d-7cfef1b14b2a","timestamp":1736911356160,"content":{"code":0,"msg":"server=s0634dc6-2;status=NP"}}]}', '{"notify":[{"heartbeat":"1736911366544"}]}']


In [8]:
#stop stream
streamer.stop()

RuntimeError: asyncio.run() cannot be called from a running event loop